### OK, I messed around with this and started by normalizing my duration and loudness between +/- 1.  That was something of an improvement, but by adding in jazz, that helped more.  Still not great, but something.  Too many neighbors (10) with this size dataset is problematic, and increasing the neighbor number started to favor the weighted accuracy while costing unweighted.
### 4 or 5 neighbors seems best to maximize both, while 3 is best for unweighted.

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
music = pd.DataFrame()
music['duration'] = [184, 134, 243, 186, 122, 197, 294, 382, 102, 264, 
                     205, 110, 307, 110, 397, 153, 190, 192, 210, 403,
                     164, 198, 204, 253, 234, 190, 182, 401, 376, 102]

music['loudness'] = [18, 34, 43, 36, 22, 9, 29, 22, 10, 24, 
                     20, 10, 17, 51, 7, 13, 19, 12, 21, 22,
                     16, 18, 4, 23, 34, 19, 14, 11, 37, 42]

music['bpm'] = [ 105, 90, 78, 75, 120, 110, 80, 100, 105, 60,
                  70, 105, 95, 70, 90, 105, 70, 75, 102, 100,
                  100, 95, 90, 80, 90, 80, 100, 105, 70, 65]

In [3]:
# a normalization function
def normalize(df, name, to_norm):
    current = df[to_norm]
    df[name] = (current - current.mean()) / (current.max() - current.min())

In [4]:
for col in music.columns[:2]:
    normalize(music, (col + "_norm"), col)

In [5]:
music['jazz'] = [ 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
                  0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
                  1, 1, 1, 1, 0, 0, 1, 1, 0, 0]

In [6]:
music.head()

,duration,loudness,bpm,duration_norm,loudness_norm,jazz
0,184,18,105,-0.129457,-0.082979,1
1,134,34,90,-0.295570,0.257447,0
2,243,43,78,0.066556,0.448936,0
3,186,36,75,-0.122813,0.300000,0
4,122,22,120,-0.335437,0.002128,1


In [37]:
from sklearn import neighbors

neighs = 3                     # set number of neighbors to use
weigh = "distance"             # set weighting to use

# Build our model.
knn = neighbors.KNeighborsRegressor(n_neighbors=neighs)
X = pd.DataFrame(music[["duration_norm", "loudness_norm", "jazz"]])

Y = music.bpm
knn.fit(X, Y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=3, p=2,
          weights='uniform')

In [38]:
# Run the same model, this time with weights. X and Y remain same as above.
knn_w = neighbors.KNeighborsRegressor(n_neighbors=neighs, weights=weigh)
knn_w.fit(X, Y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=3, p=2,
          weights='distance')

In [39]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(knn, X, Y, cv=neighs)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: 0.46 (+/- 0.22)
Weighted Accuracy: 0.20 (+/- 0.83)
